# PRA Analysis Agent demo

**Scaling analysis to get insights into the hands of PRA supervisors more quickly after earnings**

This notebook creates an analysis agent using Open AIs — GPT-4.1-nano model to synthesise the teams analysis into a concise summary for PRA Supervisors to showcase potential areas of risk discussed in the last two earnings calls of 2O25 and highlight some areas to watch for Q3. 

The analysis threads brought together cover our deep dives into identifying the themes (topics) discussed, mapping these to PRA risk categories, assessing the associated sentimemnt of each risk category and where bankers and analyst sentiment diverge, before highlighting where G-SIB bankers may have been evasive in their answers for each bank and in comparison to each other. 

This notebook: 
- Reads team csv outputs in ../data/
- Prepares them for the agent to focus on Q1 & Q2 2025 for JP Morgan + HSBC
- The agent produces a PRA-aligned narrative (Thematic Focus, Sentiment, Summarisation & Evasion, Benchmarking)
- Outputs: markdown strings + .md files in ./_outputs

## Load libraries and model

In [5]:
import os, json, glob
from pathlib import Path
import pandas as pd
from datetime import datetime
print("Key visible:", bool(os.environ.get("OPENAI_API_KEY")))  # should be True

Key visible: True


In [4]:
from openai import OpenAI
from openai import RateLimitError, APIStatusError

client = OpenAI()
MODEL = "gpt-4.1-nano"

def call_o4(input_text):
    try:
        return client.responses.create(model=MODEL, input=input_text, max_output_tokens=64)
    except RateLimitError as e:
        # 429 can mean rate-limit OR insufficient_quota; inspect the message
        msg = str(e).lower()
        if "insufficient_quota" in msg or "check your plan and billing" in msg:
            raise SystemExit("❗No API credit/budget on this project. Add billing/credits and retry.")
        else:
            # normal rate limiting — backoff and retry
            import time
            time.sleep(2)
            return client.responses.create(model=MODEL, input=input_text, max_output_tokens=64)
    except APIStatusError as e:
        raise SystemExit(f"API error {e.status_code}: {e}")

print(call_o4("ok").output_text.strip())

Hello! How can I assist you today?


## Define configurations

In [23]:
# --- CONFIG ---
DATA_DIR = Path("./data")              # adjust if needed
OUT_DIR  = Path("./_outputs"); OUT_DIR.mkdir(exist_ok=True)

BANKS = ["jpm", "hsbc"]
FOCUS_YEAR = 2025
FOCUS_QS = {"Q1","Q2"}

# Explicit filenames for this run
TOPIC_FILES = [
    DATA_DIR / "jpm_pra_topics_23_25.csv",
    DATA_DIR / "hsbc_pra_topics_23_25.csv",
]

SENTI_AGG_FILE = DATA_DIR / "sentiment_PRA_aggregated.csv"
SENTI_QNA_FILE = DATA_DIR / "sentiment_with_PRA_bank_labels_deduped.csv"

EVASION_FILES = [
    DATA_DIR / "jpm_2025_evasion_predictions_pra.csv",
    DATA_DIR / "hsbc_2025_evasion_predictions_pra.csv",
]

## Load output CSVs from Data Science Analysis

In [31]:
# --- LOAD ---
def load_topics():
    dfs = []
    for f in TOPIC_FILES:
        df = pd.read_csv(f)
        if 'bank' not in df.columns:
            bank = "JPM" if "jpm" in f.name.lower() else "HSBC"
            df['bank'] = bank
        if 'quarter' in df.columns:
            df['quarter'] = (
                df['quarter'].astype(str)
                  .str.upper()
                  .str.replace(" ", "")
                  .str.replace("INTERIM","Q2")
                  .str.replace("ANNUAL","Q4")
            )
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

def load_sentiment():
    agg = pd.read_csv(SENTI_AGG_FILE)
    qna = pd.read_csv(SENTI_QNA_FILE)
    for d in (agg, qna):
        if 'bank' in d.columns:
            d['bank'] = d['bank'].str.upper()
        if 'quarter' in d.columns:
            d['quarter'] = d['quarter'].astype(str).str.upper().str.replace(" ", "")
    return agg, qna

def load_evasion(files):
    dfs = []
    for f in files:
        bank = "JPM" if "jpm" in f.name.lower() else "HSBC"
        df = pd.read_csv(f)
        df['bank'] = bank
        if 'quarter' in df.columns:
            df['quarter'] = df['quarter'].astype(str).str.upper().str.replace(" ", "")
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)


topics = load_topics()
sent_agg, sent_qna = load_sentiment()
evasion = load_evasion([
    DATA_DIR / "jpm_2025_evasion_predictions_pra.csv",
    DATA_DIR / "hsbc_2025_evasion_predictions_pra.csv",
])

## Prepare data for the LLM agent

In [33]:
# --- FILTER: Q1 & Q2 2025 latest per bank ---
def filt_q1_q2_2025(df):
    ymask = df['year'].astype(int).eq(FOCUS_YEAR) if 'year' in df.columns else True
    qmask = df['quarter'].isin(FOCUS_QS) if 'quarter' in df.columns else True
    return df[ymask & qmask].copy()

T24 = filt_q1_q2_2025(topics)
SA24 = filt_q1_q2_2025(sent_agg)
SQ24 = filt_q1_q2_2025(sent_qna)
E24  = filt_q1_q2_2025(evasion)

In [34]:
# --- AGGREGATIONS we’ll pass to the LLM (keep it tiny & factual) ---
def thematic_summary(df):
    if df.empty: 
        return []
    # dominant topics by PRA category and topic_label
    g = (df.groupby(['bank','pra_category','topic_label'], dropna=False)
           .size()
           .reset_index(name='count'))
    top = (g.sort_values(['bank','pra_category','count'], ascending=[True,True,False])
             .groupby(['bank','pra_category'])
             .head(3))
    # roll-up by PRA category to rank categories overall
    cats = (df.groupby(['bank','pra_category']).size()
              .reset_index(name='qna_count')
              .sort_values(['bank','qna_count'], ascending=[True,False]))
    return {
        "top_topics_per_category": top.to_dict(orient="records"),
        "category_rank": cats.to_dict(orient="records")
    }

def sentiment_divergence(agg_df, qna_df, bank_name=None):
    out = {}
    if not agg_df.empty:
        piv = (
            agg_df.groupby(['pra_category','role'])[['negative','neutral','positive']]
            .mean()
            .reset_index()
        )
        if bank_name:
            piv['bank'] = bank_name.upper()
        out["role_divergence"] = piv.to_dict(orient="records")
    if not qna_df.empty:
        cols = [c for c in qna_df.columns if c.startswith("sentiment_finbert")]
        keep = ['bank','pra_category','quarter'] + cols
        out["qna_samples"] = qna_df[keep].head(100).to_dict(orient="records")
    return out

def evasion_summary(ev):
    if ev.empty: return {}
    # Expect columns like: 'evasion_score' or 'evasion_label'/'is_evasive'
    score_col = next((c for c in ev.columns if c.lower() in ["evasion_score","evasive_score","evasion_prob","ev_prob"]), None)
    label_col = next((c for c in ev.columns if c.lower() in ["evasion_label","is_evasive","evasive_flag"]), None)

    agg = []
    for bank in sorted(ev['bank'].unique()):
        sub = ev[ev['bank']==bank]
        by_cat = sub.groupby(sub.get('pra_category','Unmapped')).agg(
            qna_count=('bank','count'),
            avg_score=(score_col,'mean') if score_col in sub.columns else ('bank','count'),
            evasive_rate=((label_col, lambda x: pd.Series(x).astype(str).str.lower().isin(['1','true','yes','evasive']).mean())
                          if label_col in sub.columns else ('bank','count'))
        )
        by_cat = by_cat.reset_index(names='pra_category')
        top_evasive = by_cat.sort_values(['avg_score' if score_col else 'qna_count'], ascending=False).head(5)
        agg.append({"bank": bank, "by_category": by_cat.to_dict(orient="records"),
                    "top_evasive": top_evasive.to_dict(orient="records")})
    return {"evasion_by_bank": agg}


In [47]:
# --- TRENDS: Q1 -> Q2 deltas ---

def topic_trends(df):
    if df.empty: return {}
    trend = (
        df.groupby(['bank','pra_category','quarter'])
          .size().reset_index(name='count')
    )
    # pivot to Q1 vs Q2
    pivot = trend.pivot(index=['bank','pra_category'], columns='quarter', values='count').fillna(0)
    if "Q1" in pivot and "Q2" in pivot:
        pivot['delta_Q2vsQ1'] = pivot['Q2'] - pivot['Q1']
    return pivot.reset_index().to_dict(orient="records")

def sentiment_trends(qna_df):
    if qna_df.empty: return {}
    # avg sentiment score per role x quarter x category
    out = (
        qna_df.groupby(['bank','pra_category','quarter','sentiment_finbert_label'])
              ['sentiment_finbert_score']
              .mean()
              .reset_index()
    )
    # pivot to Q1/Q2
    q1 = out[out['quarter']=="Q1"]
    q2 = out[out['quarter']=="Q2"]
    merged = pd.merge(
        q1, q2,
        on=['bank','pra_category','sentiment_finbert_label'],
        how='outer',
        suffixes=('_Q1','_Q2')
    ).fillna(0)
    merged['delta_Q2vsQ1'] = merged['sentiment_finbert_score_Q2'] - merged['sentiment_finbert_score_Q1']
    return merged.to_dict(orient="records")

def evasion_trends(ev):
    if ev.empty: return {}
    score_col = next((c for c in ev.columns if "evasion_score" in c.lower()), None)
    label_col = next((c for c in ev.columns if "evasion_label" in c.lower()), None)

    if not score_col: return {}
    by_q = (
        ev.groupby(['bank','pra_category','quarter'])[score_col]
          .mean()
          .reset_index(name='avg_evasion')
    )
    q1 = by_q[by_q['quarter']=="Q1"]
    q2 = by_q[by_q['quarter']=="Q2"]
    merged = pd.merge(
        q1, q2,
        on=['bank','pra_category'],
        how='outer',
        suffixes=('_Q1','_Q2')
    ).fillna(0)
    merged['delta_Q2vsQ1'] = merged['avg_evasion_Q2'] - merged['avg_evasion_Q1']
    return merged.to_dict(orient="records")


In [53]:
# --- Q3 WATCH-OUTS (deterministic, from internal deltas only) ---

def _norm_label(x):
    return str(x).strip().lower()

def _top_increases_by_bank(records, bank_key='bank', cat_key='pra_category', delta_key='delta_Q2vsQ1', topn=5):
    import pandas as pd
    if not records: 
        return {}
    df = pd.DataFrame(records)
    if df.empty or bank_key not in df or cat_key not in df or delta_key not in df:
        return {}
    df = df.copy()
    # Only positive increases
    df = df[df[delta_key] > 0]
    out = {}
    for bank, sub in df.groupby(bank_key):
        top = sub.sort_values(delta_key, ascending=False).head(topn)
        out[bank] = top[[cat_key, delta_key]].to_dict(orient='records')
    return out

def build_q3_watchouts_md(topic_trend_records, sentiment_trend_records, evasion_trend_records, topn=5):
    import pandas as pd

    # --- Topics: categories with biggest Q2 rise in mentions
    topics_top = _top_increases_by_bank(topic_trend_records, topn=topn)

    # --- Evasion: categories with biggest Q2 rise in avg evasion score
    evasion_top = _top_increases_by_bank(evasion_trend_records, topn=topn)

    # --- Sentiment: biggest increase in NEGATIVE scores (FinBERT)
    neg_df = pd.DataFrame(sentiment_trend_records) if sentiment_trend_records else pd.DataFrame()
    senti_top = {}
    if not neg_df.empty:
        # robust label filter (negative / NEGATIVE)
        if 'sentiment_finbert_label' in neg_df.columns:
            neg_df = neg_df[neg_df['sentiment_finbert_label'].astype(str).str.lower() == 'negative']
        # keep only positive delta (Q2 more negative than Q1)
        if 'delta_Q2vsQ1' in neg_df.columns:
            neg_df = neg_df[neg_df['delta_Q2vsQ1'] > 0]
        if not neg_df.empty:
            for bank, sub in neg_df.groupby('bank'):
                top = sub.sort_values('delta_Q2vsQ1', ascending=False).head(topn)
                senti_top[bank] = top[['pra_category','delta_Q2vsQ1']].to_dict(orient='records')

    # --- Render markdown
    def render_bucket(title, d):
        if not d: 
            return f"- _No clear {title.lower()} identified from Q1→Q2 deltas._"
        lines = []
        for bank in sorted(d.keys()):
            lines.append(f"**{bank}**")
            for row in d[bank]:
                cat = row.get('pra_category','(Unmapped)')
                delta = row.get('delta_Q2vsQ1', 0)
                lines.append(f"- {cat}: **+{delta:.0f}**")
        return "\n".join(lines)

    md = []
    md.append("## Q3 Watch-Outs (grounded in Q1→Q2 deltas)")
    md.append("")
    md.append("**Where to expect continued scrutiny next quarter, based solely on internal quarter-to-quarter movement:**")
    md.append("")
    md.append("### 1) Rising Topic Intensity (more analyst Qs)")
    md.append(render_bucket("Topic increases", topics_top))
    md.append("")
    md.append("### 2) Rising Evasion Risk (higher avg evasion in Q2)")
    md.append(render_bucket("Evasion increases", evasion_top))
    md.append("")
    md.append("### 3) More Negative Tone (FinBERT, Q2 > Q1)")
    md.append(render_bucket("Negative sentiment increases", senti_top))
    md.append("")
    md.append("_Use these deltas to plan targeted Q3 probes (e.g., request more granularity where topic intensity and evasion both rose)._")
    return "\n".join(md)



## Setup context payload for the analysis agent

In [48]:
sentiment_payload = {
    "JPM": sentiment_divergence(SA24, SQ24[SQ24['bank']=="JPM"], bank_name="JPM"),
    "HSBC": sentiment_divergence(SA24, SQ24[SQ24['bank']=="HSBC"], bank_name="HSBC")
}

facts_payload = {
    "timebox": {"year": FOCUS_YEAR, "quarters": sorted(FOCUS_QS)},
    "thematics": thematic_summary(T24),
    "sentiment": sentiment_payload,
    "evasion": evasion_summary(E24),
    "trends": {
        "topics": topic_trends(T24),
        "sentiment": sentiment_trends(SQ24),
        "evasion": evasion_trends(E24),
    }
}

## Prepare prompt with focused instructions

In [49]:
# --- PROMPT: tightly-scoped & schema’d for a PRA narrative ---
SYSTEM = (
    "You are a concise regulatory analyst. Draft PRA-ready intelligence from structured facts. "
    "Organize by: 1) Thematic Focus, 2) Sentiment Analysis, 3) Summarisation & Evasion, 4) Benchmarking. "
    "Then produce a forward-looking 'Q3 Watch-Outs' section **grounded in quarter-on-quarter trends** "
    "(topic frequency deltas, sentiment shifts, evasion increases). "
    "Highlight categories with the sharpest changes. "
    "Avoid speculation beyond provided facts."
)

USER_INSTRUCTIONS = (
    "Synthesize the findings for Q1–Q2 2025 for JPM and HSBC using the provided facts. "
    "Map themes to PRA categories; highlight repeated risks; compare banker vs analyst tone; "
    "summarize evasiveness hot-spots with examples of categories where evasion is highest; "
    "and benchmark the two banks (systemic vs firm-specific divergences). "
    "Keep it < 600 words. Output valid GitHub-flavored Markdown."
)

def call_llm(model, system, user, facts):
    resp = client.responses.create(
        model=model,
        input=[
            {"role": "system", "content": system},
            {"role": "user", "content": user},
            {"role": "user", "content": "Facts JSON:\n" + json.dumps(facts, ensure_ascii=False, separators=(",", ":"))}
        ],
        max_output_tokens=900,
        temperature=0.2,
    )
    # navigate to text output
    return resp.output[0].content[0].text

## Run analysis, saving markdown report

In [56]:
# Build the deterministic watch-outs section from the existing trend outputs
q3_watchouts_md = build_q3_watchouts_md(
    topic_trend_records=facts_payload["trends"]["topics"],
    sentiment_trend_records=facts_payload["trends"]["sentiment"],
    evasion_trend_records=facts_payload["trends"]["evasion"],
    topn=5
)

report_md = call_llm(MODEL, SYSTEM, USER_INSTRUCTIONS, facts_payload)

# 1) Option A — append to model output (guarantees the section appears)
report_md = report_md.rstrip() + "\n\n" + q3_watchouts_md + "\n"

In [57]:
# --- Save & show ---
stamp = datetime.now().strftime("%Y%m%d-%H%M")
out_path = OUT_DIR / f"PRA_synthesis_Q1Q2_{FOCUS_YEAR}_{stamp}.md"
out_path.write_text(report_md, encoding="utf-8")
print(f"\n--- PRA synthesis saved: {out_path}\n")
print(report_md[:1500] + ("\n...\n" if len(report_md)>1500 else "\n"))


--- PRA synthesis saved: _outputs/PRA_synthesis_Q1Q2_2025_20251002-2010.md

# Q1–Q2 2025 Regulatory Synthesis: JPM & HSBC

## 1. Thematic Focus & PRA Categories

### HSBC
- **Capital Adequacy**: Dominant concern, with high topic counts (Q1: 7; Q2: 6). Key topics include revenue growth, impairment, and hedge structures.
- **Conduct Risk**: Significant focus, with 11 topics in Q1 decreasing to 8 in Q2. Notable concerns around trade scenarios, tariffs, and China-related downside risks.
- **Governance**: Elevated attention (Q1: 14; Q2: 10), highlighting outliers and normalised impairment guidance.
- **Liquidity & Unmapped**: Stable in liquidity topics; however, unmapped topics (e.g., costs, Mexico business) show a sharp decline from 14 to 3, indicating reduced focus or evasiveness.

### JPM
- **Governance**: The most prominent theme, with an increase from 44 to 61 topics, emphasizing governance outliers and follow-up discussions.
- **Capital Adequacy**: Consistent high topic counts (Q1: 1

# GENERATED REPORT

# Q1–Q2 2025 Regulatory Synthesis: JPM & HSBC

## 1. Thematic Focus & PRA Categories

### HSBC
- **Capital Adequacy**: Dominant concern, with high topic counts (Q1: 7; Q2: 6). Key topics include revenue growth, impairment, and hedge structures.
- **Conduct Risk**: Significant focus, with 11 topics in Q1 decreasing to 8 in Q2. Notable concerns around trade scenarios, tariffs, and China-related downside risks.
- **Governance**: Elevated attention (Q1: 14; Q2: 10), highlighting outliers and normalised impairment guidance.
- **Liquidity & Unmapped**: Stable in liquidity topics; however, unmapped topics (e.g., costs, Mexico business) show a sharp decline from 14 to 3, indicating reduced focus or evasiveness.

### JPM
- **Governance**: The most prominent theme, with an increase from 44 to 61 topics, emphasizing governance outliers and follow-up discussions.
- **Capital Adequacy**: Consistent high topic counts (Q1: 19; Q2: 42), with emphasis on Basel III, endgame, and client lending.
- **Conduct & Credit Risks**: Steady topics, with slight increases, reflecting ongoing risk assessments.
- **Unmapped**: Topics reduced from 8 to 3, but with high evasiveness, suggesting strategic opacity.

## 2. Sentiment Analysis & Divergences

### Banker vs. Analyst Tone
- **HSBC**: Banker sentiment remains highly positive (Q2: ~0.96–0.99), with minor fluctuations. Analysts exhibit more negative or neutral tones, especially in conduct and credit risk (Q1 negative scores ~0.99, Q2 dropping to 0), indicating potential evasiveness or cautiousness.
- **JPM**: Bankers are optimistic (Q2: ~0.97–0.99), while analysts show a stark contrast, with sentiment scores plummeting to zero in Q2 for several categories, notably conduct risk (from 0.96 to 0.00). This divergence suggests increased analyst skepticism or evasion.

### Sentiment Shifts
- **HSBC**: Slight positive shift in capital adequacy (+0.07), stable in other categories.
- **JPM**: Notable positive trend in governance (+0.17) and capital adequacy (+0.19), but analyst sentiment in conduct risk and market risk drops sharply, indicating potential strategic evasion or reporting gaps.

## 3. Summarisation & Evasion Hot-Spots

### Repeated Risks
- **Capital Adequacy**: Both banks exhibit high topic counts and evasion rates (~17–31), indicating persistent opacity.
- **Conduct & Credit Risks**: Elevated topic counts and evasion (~5–12), with HSBC showing increased evasiveness in credit risk (from 5 to 6) and JPM in operational controls (from 0 to 42).
- **Unmapped & Strategy**: JPM’s high evasiveness (21 topics, 21% rate) in unmapped areas and HSBC’s reduction in unmapped topics suggest strategic evasion hotspots.

### Evasion Hotspots & Examples
- **HSBC**: Highest evasion in Capital Adequacy (17%), Costs & Efficiency (6%), and Credit Risk (5%). Notably, Q2 evasion in Costs & Efficiency drops to zero, possibly indicating strategic suppression.
- **JPM**: Major evasion in Regulatory/PRA interactions (2 topics, 2%), and Operational Risk/Controls (from 0 to 42), indicating increased opacity in operational controls and regulatory engagement.

## 4. Benchmarking: Systemic vs. Firm-Specific Divergences

| Aspect | HSBC | JPM | Divergence |
|---------|--------|--------|------------|
| **Themes** | Focused on Capital Adequacy, Conduct, Governance | Dominant in Governance, Capital, and Strategy | JPM shows broader governance focus; HSBC more on capital & conduct |
| **Sentiment** | Generally positive; minor fluctuations | Similar positivity but with sharper analyst-bank divergence | JPM’s analyst skepticism more pronounced |
| **Evasion** | Higher in Capital Adequacy (~17%) and Costs (~6%) | Highest

## Q3 Watch-Outs (grounded in Q1→Q2 deltas)

**Where to expect continued scrutiny next quarter, based solely on internal quarter-to-quarter movement:**

### 1) Rising Topic Intensity (more analyst Qs)
**JPM**
- Capital Adequacy: **+23**
- Governance: **+17**
- Conduct Risk: **+2**

### 2) Rising Evasion Risk (higher avg evasion in Q2)
**HSBC**
- Credit risk: **+10**
**JPM**
- Operational risk / Controls: **+42**
- Interest rate risk / NII: **+9**
- Costs & efficiency: **+8**
- Credit risk: **+7**
- Capital adequacy: **+4**

### 3) More Negative Tone (FinBERT, Q2 > Q1)
**HSBC**
- Market risk: **+1**
- Credit risk: **+0**
**JPMORGANCHASE**
- Capital adequacy: **+1**
- Credit risk: **+0**
**RBC CAPITAL MARKETS LLC**
- Market risk: **+1**

_Use these deltas to plan targeted Q3 probes (e.g., request more granularity where topic intensity and evasion both rose)._
